In [ ]:
# Build and clean dataset
import pandas as pd
import altair as alt
import geopandas as gpd
from multiprocessing import Pool
import json
import plotly.express as px
import plotly.graph_objects as go
import pyproj
import math

Based on https://www.ncei.noaa.gov/news/noaa-offers-climate-data-counties NClimDiv

https://www.ncei.noaa.gov/pub/data/cirs/climdiv/

In [ ]:
div_weather = pd.read_parquet('climdiv-tmpcdv-v1.0.0-20230504.parquet')
div_weather['state_div'] = div_weather['state_code'] + div_weather['division_number']
div_weather.set_index('state_div', inplace=True)
div_weather.head()

In [ ]:
# Read in the shapefile of climate divisions
gdf = gpd.read_file('CONUS_CLIMATE_DIVISIONS.shp/GIS.OFFICIAL_CLIM_DIVISIONS.shp')
# gdf_converted = gdf.to_crs(pyproj.CRS.from_epsg(4326))
gdf_converted = gdf
gdf_converted['division_number'] = gdf_converted['CD_NEW'].apply(lambda x: f'{int(x):02d}0')
gdf_converted['state_code'] = gdf_converted['STATE_FIPS']
gdf_converted['state_div'] = gdf_converted['state_code'] + gdf_converted['division_number']
gdf_converted.set_index('state_div', inplace=True)
gdf_converted.plot()

In [ ]:
# # Join the weather data to the shapefile
# gdf_weather_23 = gdf_converted.merge(div_weather.query('year == "2023" and month == "01"'), on = 'state_div', how='left')
# # Create background map to fill in missing values
# foreground = alt.Chart(gdf_weather_23).mark_geoshape().encode(
#     color = 'value:Q',
#     # Add outlines
#     stroke = alt.value('black'),
# ).project(
#     type = 'albersUsa'
# ).properties(
#     width = 800,
#     height= 800
# ).add_params(
#     alt.selection_interval(bind='scales')
# )


# background = alt.Chart(gdf_weather_23).mark_geoshape(
#     color = 'lightgray',
#     stroke = 'darkgrey',
# ).project(
#     type = 'albersUsa'
# ).properties(
#     width = 800,
#     height= 800
# ).add_params(
#     alt.selection_interval(bind='scales')
# )

# chart = background + foreground
# chart

In [ ]:
# Join the weather data to the shapefile
gdf_weather = gdf_converted.merge(div_weather, on = 'state_div', how='outer')
gdf_weather_23 = gdf_converted.merge(div_weather.query('year == "2023" and month == "01"'), on = 'state_div', how='outer')

In [ ]:
gdf_weather_23.plot(column = 'value', 
                    legend = True, 
                    missing_kwds={
                        "color": "lightgrey",
                        "edgecolor": "grey",
                        "hatch": "///",
                        "label": "Missing values",},
                    figsize = (20,10))

In [ ]:
# Create bokeh map
# Import figure
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Viridis256 as palette
from bokeh.transform import linear_cmap
from bokeh.models import GeoJSONDataSource



p = figure(title = "Weather Map",
              x_axis_label = 'Longitude',
                y_axis_label = 'Latitude')




states = p.patches(
    'xs','ys',
    fill_color = 'blue',
    fill_alpha = 0.7,
    line_color = 'black',
    line_width = 0.5,
    source = GeoJSONDataSource(geojson = gdf_weather_23[['geometry','value']].to_json())
)
# show(p)

In [ ]:
# Create plotly map
# First create new variable with WGS84 CRS
gdf_weather_23_plt = gdf_weather_23
gdf_weather_23_plt['geometry'] = gdf_weather_23_plt['geometry'].to_crs("WGS84")

In [ ]:
# px.choropleth_mapbox(
#     gdf_weather_23_plt,
#     geojson = gdf_weather_23_plt.geometry,
#     locations = gdf_weather_23_plt.index,
#     color = 'value',
#     color_continuous_scale = 'Viridis',
#     mapbox_style = 'carto-positron',
#     zoom = 3
# )